In [12]:
import os
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField, SearchField,VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery
from azure.identity import DefaultAzureCredential
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv
import PyPDF2

# Load environment variables
load_dotenv()

# Azure OpenAI and Azure Search credentials
SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_version = os.getenv('API_VERSION')
azure_openai_embedding_deployment = "text-embedding-ada-002"
azure_openai_chat_model = "gpt-35-turbo"

# Constants
SEARCH_INDEX_NAME = "clinical_trial_test"

In [15]:
def search_embeddings(user_question):
    client_search = SearchClient(endpoint=SEARCH_ENDPOINT, index_name=SEARCH_INDEX_NAME, credential=AzureKeyCredential(SEARCH_ADMIN_KEY))
    # Generate embeddings for the user's question using Azure OpenAI
    try:
        openai_credential = AzureKeyCredential(azure_openai_key)
        client = AzureOpenAI(
            azure_deployment=azure_openai_embedding_deployment,
            azure_endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            api_version=azure_openai_version,
        )
        response = client.embeddings.create(input=user_question, model="text-embedding-ada-002").data[0].embedding

        vector_query = VectorizedQuery(vector=response, k_nearest_neighbors=3, fields="contentEmbeddings", exhaustive=True)
        results = client_search.search(
            search_text=None,
            vector_queries=[vector_query],
            select=["id", "content"],
            top=3
        )
        return results
    except Exception as e:
        print(f"Error generating embeddings or searching: {e}")
        return None

In [16]:
a = search_embeddings("Trial design")
print(a)

<iterator object azure.core.paging.ItemPaged at 0x26c85157c10>


In [17]:
def search_embeddings(user_question):
    client_search = SearchClient(
        endpoint=SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=AzureKeyCredential(SEARCH_ADMIN_KEY)
    )
    
    try:
        # Generate embeddings for the user's question using Azure OpenAI
        client = AzureOpenAI(
            azure_deployment=azure_openai_embedding_deployment,
            azure_endpoint=azure_openai_endpoint,
            api_key=azure_openai_key,
            api_version=azure_openai_version,
        )
        
        # Get the embeddings for the user question
        response = client.embeddings.create(
            input=user_question, 
            model="text-embedding-ada-002"
        ).data[0].embedding

        # Create a vector search query
        search_results = client_search.search(
            search_text="",  # Empty string for vector search
            vectors=[{
                "value": response,  # Pass the embedding vector directly
                "fields": "contentEmbeddings",  # Field to search against
                "k": 3  # Number of nearest neighbors to retrieve
            }],
            top=3,  # Limit results to top 3
            select=["id", "content"]
        )

        # If no results are found, return None
        if not search_results:
            print(f"No search results found for the query: {user_question}")
            return None

        # Extract and return the search results
        results = [
            {
                "id": result["id"],
                "content": result["content"]
            } 
            for result in search_results
        ]
        
        return results

    except Exception as e:
        print(f"Error in search_embeddings: {e}")
        return None

# Example usage
a = search_embeddings("Trial design")
print(a)


Error in search_embeddings: Session.request() got an unexpected keyword argument 'vectors'
None
